# PCA and k-means

## Setting up

In [ ]:
"""
Author      : Yi-Chieh Wu, Sriram Sankararaman
"""

# numpy and scipy libraries
import numpy as np
from scipy import stats

# matplotlib libraries
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
# To add your own Drive Run this cell.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
# Change the path below to the path where your folder locates
# where you have util.py
### ========== TODO : START ========== ###
sys.path.append('/content/gdrive/My Drive/CS M146 Data')
### ========== TODO : START ========== ###


In [ ]:
import util
from util import *

## Point, Cluster and Set of Clusters classes

In [ ]:
######################################################################
# classes
######################################################################

class Point(object) :
    
    def __init__(self, name, label, attrs) :
        """
        A data point.
        
        Attributes
        --------------------
            name  -- string, name
            label -- string, label
            attrs -- string, features
        """
        
        self.name = name
        self.label = label
        self.attrs = attrs
    
    
    #============================================================
    # utilities
    #============================================================
    
    def distance(self, other) :
        """
        Return Euclidean distance of this point with other point.
        
        Parameters
        --------------------
            other -- Point, point to which we are measuring distance
        
        Returns
        --------------------
            dist  -- float, Euclidean distance
        """
        # Euclidean distance metric
        return np.linalg.norm(self.attrs-other.attrs)
    
    
    def __str__(self) :
        """
        Return string representation.
        """
        return "%s : (%s, %s)" % (self.name, str(self.attrs), self.label)

In [ ]:
class Cluster(object) :
    
    def __init__(self, points) :
        """
        A cluster (set of points).
        
        Attributes
        --------------------
            points -- list of Points, cluster elements
        """        
        self.points = points
    
    
    def __str__(self) :
        """
        Return string representation.
        """
        s = ""
        for point in self.points :
            s += str(point)
        return s
        
    #============================================================
    # utilities
    #============================================================
    
    def purity(self) :
        """
        Compute cluster purity.
        
        Returns
        --------------------
            n           -- int, number of points in this cluster
            num_correct -- int, number of points in this cluster
                                with label equal to most common label in cluster
        """        
        labels = []
        for p in self.points :
            labels.append(p.label)
        
        cluster_label, count = stats.mode(labels)
        return len(labels), np.float64(count)
    
    
    def centroid(self) :
        """
        Compute centroid of this cluster.
        
        Returns
        --------------------
            centroid -- Point, centroid of cluster
        """
        
        ### ========== TODO : START ========== ###
        # part 2b: implement
        # set the centroid label to any value (e.g. the most common label in this cluster)
        att = 0
        dictionary = {}
        sortedDict = {}
        for i in self.points:
          if i.label not in dictionary:
            dictionary[i.label] = 1
          else:
            dictionary[i.label] += 1
          att = att + i.attrs
        sortdict = sorted(dictionary, key=dictionary.get)
        for k in sortdict:
          sortedDict[k] = dictionary[k]
        att = att/len(self.points)
        commonLabel = list(sortedDict.keys())[-1]
        centroid = Point('centroid', commonLabel, att)
        return centroid
        ### ========== TODO : END ========== ###
    
    
    def medoid(self) :
        """
        Compute medoid of this cluster, that is, the point in this cluster
        that is closest to all other points in this cluster.
        
        Returns
        --------------------
            medoid -- Point, medoid of this cluster
        """
        
        ### ========== TODO : START ========== ###
        # part 2b: implement
        closest = None
        bestSoFar = sys.float_info.max
        for i in self.points:
          dis = 0
          for j in self.points:
            dis = dis + i.distance(j)
          if dis <= bestSoFar:
            bestSoFar = dis
            closest = i 
        medoid = closest
        return medoid
        ### ========== TODO : END ========== ###
    
    
    def equivalent(self, other) :
        """
        Determine whether this cluster is equivalent to other cluster.
        Two clusters are equivalent if they contain the same set of points
        (not the same actual Point objects but the same geometric locations).
        
        Parameters
        --------------------
            other -- Cluster, cluster to which we are comparing this cluster
        
        Returns
        --------------------
            flag  -- bool, True if both clusters are equivalent or False otherwise
        """
        
        if len(self.points) != len(other.points) :
            return False
        
        matched = []
        for point1 in self.points :
            for point2 in other.points :
                if point1.distance(point2) == 0 and point2 not in matched :
                    matched.append(point2)
        return len(matched) == len(self.points)

In [ ]:
class ClusterSet(object):

    def __init__(self) :
        """
        A cluster set (set of clusters).
        
        Parameters
        --------------------
            members -- list of Clusters, clusters that make up this set
        """
        self.members = []
    
    
    #============================================================
    # utilities
    #============================================================    
    
    def centroids(self) :
        """
        Return centroids of each cluster in this cluster set.
        
        Returns
        --------------------
            centroids -- list of Points, centroids of each cluster in this cluster set
        """
        
        ### ========== TODO : START ========== ###
        # part 2b: implement
        centroids = []
        for i in self.members:
          centroids.append(i.centroid())
        return centroids
        ### ========== TODO : END ========== ###
    
    
    def medoids(self) :
        """
        Return medoids of each cluster in this cluster set.
        
        Returns
        --------------------
            medoids -- list of Points, medoids of each cluster in this cluster set
        """
        
        ### ========== TODO : START ========== ###
        # part 2b: implement
        medoids = []
        for i in self.members:
          medoids.append(i.medoid())
        return medoids
        ### ========== TODO : END ========== ###
    
    
    def score(self) :
        """
        Compute average purity across clusters in this cluster set.
        
        Returns
        --------------------
            score -- float, average purity
        """
        
        total_correct = 0
        total = 0
        for c in self.members :
            n, n_correct = c.purity()
            total += n
            total_correct += n_correct
        return total_correct / float(total)
    
    
    def equivalent(self, other) :
        """ 
        Determine whether this cluster set is equivalent to other cluster set.
        Two cluster sets are equivalent if they contain the same set of clusters
        (as computed by Cluster.equivalent(...)).
        
        Parameters
        --------------------
            other -- ClusterSet, cluster set to which we are comparing this cluster set
        
        Returns
        --------------------
            flag  -- bool, True if both cluster sets are equivalent or False otherwise
        """
        
        if len(self.members) != len(other.members):
            return False
        
        matched = []
        for cluster1 in self.members :
            for cluster2 in other.members :
                if cluster1.equivalent(cluster2) and cluster2 not in matched:
                    matched.append(cluster2)
        return len(matched) == len(self.members)
    
    
    #============================================================
    # manipulation
    #============================================================

    def add(self, cluster):
        """
        Add cluster to this cluster set (only if it does not already exist).
        
        If the cluster is already in this cluster set, raise a ValueError.
        
        Parameters
        --------------------
            cluster -- Cluster, cluster to add
        """
        
        if cluster in self.members :
            raise ValueError
        
        self.members.append(cluster)

## k-means and k-medoids algorithms

In [ ]:
######################################################################
# k-means and k-medoids
######################################################################

def random_init(points, k) :
    """
    Randomly select k unique elements from points to be initial cluster centers.
    
    Parameters
    --------------------
        points         -- list of Points, dataset
        k              -- int, number of clusters
    
    Returns
    --------------------
        initial_points -- list of k Points, initial cluster centers
    """
    ### ========== TODO : START ========== ###
    # part 2c: implement (hint: use np.random.choice)
    initial_points = np.random.choice(points,k,replace=False)
    return initial_points
    ### ========== TODO : END ========== ###


def cheat_init(points) :
    """
    Initialize clusters by cheating!
    
    Details
    - Let k be number of unique labels in dataset.
    - Group points into k clusters based on label (i.e. class) information.
    - Return medoid of each cluster as initial centers.
    
    Parameters
    --------------------
        points         -- list of Points, dataset
    
    Returns
    --------------------
        initial_points -- list of k Points, initial cluster centers
    """
    ### ========== TODO : START ========== ###
    # part 2f: implement
    dictPts = {}
    for i in points:
      Arr = []
      if i.label not in dictPts:
        temp = [i]
        dictPts[i.label] = temp
      else:
        dictPts[i.label].append(i)
    clustArr = []
    for key,value in dictPts.items():
      clustArr.append(Cluster(value))
    initial_points = []
    for j in clustArr:
      initial_points.append(j.medoid())
    return initial_points
    ### ========== TODO : END ========== ###


def kMeans(points, k, init='random', plot=False) :
    """
    Cluster points into k clusters using variations of k-means algorithm.
    
    Parameters
    --------------------
        points  -- list of Points, dataset
        k       -- int, number of clusters
        average -- method of ClusterSet
                   determines how to calculate average of points in cluster
                   allowable: ClusterSet.centroids, ClusterSet.medoids
        init    -- string, method of initialization
                   allowable: 
                       'cheat'  -- use cheat_init to initialize clusters
                       'random' -- use random_init to initialize clusters
        plot    -- bool, True to plot clusters with corresponding averages
                         for each iteration of algorithm
    
    Returns
    --------------------
        k_clusters -- ClusterSet, k clusters
    """
    
    ### ========== TODO : START ========== ###
    # part 2c: implement
    # Hints:
    #   (1) On each iteration, keep track of the new cluster assignments
    #       in a separate data structure. Then use these assignments to create
    #       a new ClusterSet object and update the centroids.
    #   (2) Repeat until the clustering no longer changes.
    #   (3) To plot, use plot_clusters(...).
    k_clusters = kAverages(points=points,k=k,average = 'centroids',init= init,plot = plot)
    return k_clusters
    ### ========== TODO : END ========== ###


def kMedoids(points, k, init='random', plot=False) :
    """
    Cluster points in k clusters using k-medoids clustering.
    See kMeans(...).
    """
    ### ========== TODO : START ========== ###
    # part 2e: implement

    k_clusters = kAverages(points=points,k=k,average = 'medoids',init= init,plot = plot)
    return k_clusters
    ### ========== TODO : END ========== ###
def kAverages(points, k, average, init='random', plot=True):
    ind = 0;
    pts = None
    ptSoFar = None
    pastPts = None
    if init == 'random': 
        pts = random_init(points, k)
    else:
        pts = cheat_init(points)
    #run unil converge
    for i in iter(int,1):
      ind = ind+1
      ptSoFar = ClusterSet()
      #assign points to clusters
      dictPts = helper(points=points,cent=pts)
      for key,value in dictPts.items():
        ptSoFar.add(Cluster(value))
      if plot == True:
        titStr = init + " Initialization " + average + " Clustering" + " Iteration " + str(ind)
        if average == 'centroids':
          plot_clusters(ptSoFar, titStr, ClusterSet.centroids)
        else:
          plot_clusters(ptSoFar, titStr, ClusterSet.medoids)
      if pastPts != None and ptSoFar.equivalent(pastPts):
        return ptSoFar
      else:
        #update the pastpts to current cluster
        pastPts = ptSoFar
        if average == 'centroids':
          pts = ptSoFar.centroids()
        else:
          pts = ptSoFar.medoids()
#Returns a dictionary with key being the index of different centroids
#Corresponding Values are an array of points
def helper(points, cent):
    dictPts = {}
    for i in points:
      distArr = []
      for c in cent:
        distArr.append(i.distance(c))
      mIndex = distArr.index(np.min(distArr))
      if mIndex not in dictPts:
        temp = [i]
        dictPts[mIndex] = temp
      else:
        dictPts[mIndex].append(i)
    return dictPts




## Utilities

In [ ]:
######################################################################
# helper functions
######################################################################

def build_face_image_points(X, y) :
    """
    Translate images to (labeled) points.
    
    Parameters
    --------------------
        X     -- numpy array of shape (n,d), features (each row is one image)
        y     -- numpy array of shape (n,), targets
    
    Returns
    --------------------
        point -- list of Points, dataset (one point for each image)
    """
    
    n,d = X.shape
    
    images = defaultdict(list) # key = class, val = list of images with this class
    for i in range(n) :
        images[y[i]].append(X[i,:])
    
    points = []
    for face in images :
        count = 0
        for im in images[face] :
            points.append(Point(str(face) + '_' + str(count), face, im))
            count += 1

    return points


def plot_clusters(clusters, title, average) :
    """
    Plot clusters along with average points of each cluster.

    Parameters
    --------------------
        clusters -- ClusterSet, clusters to plot
        title    -- string, plot title
        average  -- method of ClusterSet
                    determines how to calculate average of points in cluster
                    allowable: ClusterSet.centroids, ClusterSet.medoids
    """
    
    plt.figure()
    np.random.seed(20)
    label = 0
    colors = {}
    centroids = average(clusters)
    for c in centroids :
        coord = c.attrs
        plt.plot(coord[0],coord[1], 'ok', markersize=12)
    for cluster in clusters.members :
        label += 1
        colors[label] = np.random.rand(3,)
        for point in cluster.points :
            coord = point.attrs
            plt.plot(coord[0], coord[1], 'o', color=colors[label])
    plt.title(title)
    plt.show()


def generate_points_2d(N, seed=1234) :
    """
    Generate toy dataset of 3 clusters each with N points.
    
    Parameters
    --------------------
        N      -- int, number of points to generate per cluster
        seed   -- random seed
    
    Returns
    --------------------
        points -- list of Points, dataset
    """
    np.random.seed(seed)
    
    mu = [[0,0.5], [1,1], [2,0.5]]
    sigma = [[0.1,0.1], [0.25,0.25], [0.15,0.15]]
    
    label = 0
    points = []
    for m,s in zip(mu, sigma) :
        label += 1
        for i in range(N) :
            x = random_sample_2d(m, s)
            points.append(Point(str(label)+'_'+str(i), label, x))
    
    return points

## Main function

In [ ]:
######################################################################
# main
######################################################################

def main() :
    ### ========== TODO : START ========== ###
    # part 1: explore LFW data set
    x, y = get_lfw_data()
    n, d = x.shape
    for k in range(5):
      show_image(im=x[k])
    aveImg = np.mean(x, axis = 0)
    show_image(im = aveImg)
    U, mu = util.PCA(x)
    plot_gallery([vec_to_image(U[:,i]) for i in range(12)])

    L = [1, 10, 50, 100, 500, 1288]
    for i in L:
      a, b = apply_PCA_from_Eig(X=x,U=U,l=i,mu=mu)
      high = reconstruct_from_PCA(Z=a,U=b,mu=mu)
      print("Reconstruct with l = ", i)
      plot_gallery(high)
    ### ========== TODO : END ========== ###
    
    
    
    ### ========== TODO : START ========== ###
    # part 2d-2f: cluster toy dataset
    np.random.seed(1234)
    pts = generate_points_2d(N=20)
    kMeans(points = pts, k=3, init="random", plot=True)
    kMedoids(points = pts, k=3, init="random", plot=True)
    kMeans(points = pts, k=3, init="cheat", plot=True)
    kMedoids(points = pts, k=3, init="cheat", plot=True)
    ### ========== TODO : END ========== ###
    
    
    
    ### ========== TODO : START ========== ###    
    # part 3a: cluster faces
    np.random.seed(1234)
    X1, y1 = util.limit_pics(x, y, [4, 6, 13, 16], 40)
    points = build_face_image_points(X1, y1)
    KmeanScore = []
    KmedoidScore = []
    for i in range(10):
      kmeanClust = kMeans(points, 4, init="random", plot=False)
      kmedoidClust = kMedoids(points, 4, init="random", plot=False)
      KmeanScore.append(kmeanClust.score())
      KmedoidScore.append(kmedoidClust.score())
    print("k-means average = ", np.mean(KmeanScore))
    print("k-means min = ", np.min(KmeanScore))
    print("k-means max = ", np.max(KmeanScore))
    print("k-medoids average = ", np.mean(KmedoidScore))
    print("k-medoids min = ", np.min(KmedoidScore))
    print("k-medoids max = ", np.max(KmedoidScore))
    # part 3b: explore effect of lower-dimensional representations on clustering performance
    np.random.seed(1234)
    U, mu = util.PCA(x)
    l = [] 
    for i in range(42):
       if i % 2 != 0:
         l.append(i)
    X2, y2 = util.limit_pics(x, y, [4, 13], 40)
    KmeanLScore = []
    KmedoidLScore = []
    for k in l:
      Z, Ul = util.apply_PCA_from_Eig(X2, U, k, mu)
      Xrecons = util.reconstruct_from_PCA(Z, Ul, mu)
      points = build_face_image_points(Xrecons, y2)
      kmeanLClust = kMeans(points, 2, init="cheat", plot=False)
      kmedoidLClust = kMedoids(points, 2, init="cheat", plot=False)
      KmeanLScore.append(kmeanLClust.score())
      KmedoidLScore.append(kmedoidLClust.score())
    plt.figure()
    plt.plot(l, KmeanLScore)
    plt.plot(l, KmedoidLScore)
    plt.legend(["K-Means", "K-Medoids"])
    plt.title("Score vs. l Plot")
    # part 3c: determine ``most discriminative'' and ``least discriminative'' pairs of images
    np.random.seed(1234)
    minIndex = [-1 , -1]
    maxIndex = [-1 , -1]
    minScore = sys.float_info.max
    maxScore = sys.float_info.min
    for m in range(19):
      for n in range(19):
        if m == n:
          continue
        X3, y3 = util.limit_pics(x, y, [m, n], 40)
        points = build_face_image_points(X3, y3)
        kmeanClust = kMeans(points, 2, init="cheat", plot=False)
        score = kmeanClust.score()
        if score < minScore:
          minScore = score
          minIndex[0] = m
          minIndex[1] = n
        if score > maxScore:
          maxScore = score
          maxIndex[0] = m
          maxIndex[1] = n
    plot_representative_images(x, y, [minIndex[0], minIndex[1]], "Min Score")
    plot_representative_images(x, y, [maxIndex[0], maxIndex[1]], "Max Score")






    ### ========== TODO : END ========== ###


if __name__ == "__main__" :
    main()